In [1]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm

## Part II: Estimation of Sensitivities in MC

In [2]:
def Delta_Analytical_Call(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

def Delta_Analytical_Put(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1) - math.e**-r

In [6]:
@jit(nopython=True, fastmath=True, parallel=True)
def European_Put(T, K, r, S, sigma, trials):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if K-S_cur > 0:
            payoff_array[i] = (K-S_cur)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


@jit(nopython=True, fastmath=True, parallel=True)
def European_Call(T, K, r, S, sigma, trials):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur-K > 0:
            payoff_array[i] = (S_cur-K)*np.exp(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array


@jit(nopython=True, fastmath=True, parallel=True)
def Digital_Call(T, K, r, S, sigma, trials):
    
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    payoff_array = np.zeros(trials)
    
    for i in range(trials):
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*np.random.normal())
        
        if S_cur > K:
            payoff_array[i] = 1 * math.e**(-r*T)
        else:
            payoff_array[i] = 0

    return payoff_array
    

def get_delta_put(kwargs, S, e, seed=None):
    if seed:
        np.random.seed(seed)
    kwargs['S'] = S
    V = European_Put(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = European_Put(**kwargs)
    
    return (Ve-V)/ e


def get_delta_call(kwargs, S, e, seed=None):
    if seed:
        np.random.seed(seed)
        
    kwargs['S'] = S
    V = European_Call(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = European_Call(**kwargs)
    
    return (Ve-V)/ e


def get_delta_digital(kwargs, S, e, seed=None):
    if seed:
        #print("im here")
        np.random.seed(seed)
        
    kwargs['S'] = S
    V = Digital_Call(**kwargs)
    
    
    kwargs['S'] = S + e
    if seed:
        np.random.seed(seed)
    Ve = Digital_Call(**kwargs)
    
    return (Ve-V)/ e

In [14]:
analytical = Delta_Analytical_Call(100, 99, 0.06, 0.2, 1)
analytical

0.6737355117348961

In [7]:
kwargs = {}
kwargs['T'] = 1
kwargs['K'] = 99
kwargs['r'] = 0.06
kwargs['sigma'] = 0.2
kwargs['trials'] = 10**6

In [9]:
S = 100
e = 0.02

delta_matrix_noseeds = [get_delta_call(kwargs, S, e, None) for x in range(100)]
delta_matrix_seeds = [get_delta_call(kwargs, S, e, 100) for x in range(100)]

In [10]:
np.mean(delta_matrix_seeds)

0.6566792376718549

In [11]:
np.std(delta_matrix_seeds) / np.sqrt(len(delta_matrix_seeds[0]) * len(delta_matrix_seeds))

0.10821337993717059

In [12]:
np.mean(delta_matrix_noseeds)

0.7517803621832106

In [ ]:
diff = np.zeros(len(delta_matrix_seeds))
for i, delta in enumerate(delta_matrix_seeds):
    diff[i] = abs(analytical - np.mean(delta))

In [ ]:
np.mean(diff)

In [ ]:
np.mean(delta_matrix_noseeds)

In [13]:
np.std(delta_matrix_noseeds) / np.sqrt(len(delta_matrix_noseeds[0]) * len(delta_matrix_noseeds))

0.10821995287370935

In [ ]:
S = 100
e = 1
delta_matrix = [get_delta_digital(kwargs, S, e, 40) for x in range(100)]

In [ ]:
np.mean(delta_matrix)